In [74]:
import pandas as pd
from hmmlearn import hmm
import numpy as np
from collections import Counter


In [75]:
df = pd.read_csv('training_dataset.csv')

grouped_data = df.groupby(['id', df.groupby('id')['laneId'].transform('first')])

data_dict = {}

for group_name, group_data in grouped_data:
    data_dict[group_name] = group_data

selected_groups2 = grouped_data.filter(lambda x: x['laneId'].iloc[0] == 2)
selected_groups3 = grouped_data.filter(lambda x: x['laneId'].iloc[0] == 3)
selected_groups5 = grouped_data.filter(lambda x: x['laneId'].iloc[0] == 5)
selected_groups6 = grouped_data.filter(lambda x: x['laneId'].iloc[0] == 6)

train_data2 = selected_groups2[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values
train_data3 = selected_groups3[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values
train_data5 = selected_groups5[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values
train_data6 = selected_groups6[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values

print(train_data2)
num_rows = len(train_data2)

print(num_rows)


[[ 1.6275e+02  9.3900e+00 -3.2040e+01  0.0000e+00  2.0000e+00]
 [ 1.6158e+02  9.3900e+00 -3.2060e+01  0.0000e+00  2.0000e+00]
 [ 1.6036e+02  9.3900e+00 -3.2070e+01  0.0000e+00  2.0000e+00]
 ...
 [ 3.1873e+02  9.3600e+00 -2.9460e+01  2.4000e-01  2.0000e+00]
 [ 3.1754e+02  9.3700e+00 -2.9450e+01  2.5000e-01  2.0000e+00]
 [ 3.1636e+02  9.3800e+00 -2.9440e+01  2.5000e-01  2.0000e+00]]
93546


In [76]:
df = pd.read_csv('train_dataset2c.csv')

train_data2c = df[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values

df = pd.read_csv('train_dataset6c.csv')

train_data6c = df[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values


In [113]:
model2c = hmm.GaussianHMM(n_components=4, covariance_type="full", n_iter=10000)

model2c.fit(train_data2c)

print("Training Finish!")

Training Finish!


In [114]:
print(model2c.means_)

[[ 2.62695385e+02  9.73051282e+00 -3.62453846e+01  3.46923077e-01
   2.00000000e+00]
 [ 4.88581579e+01  1.32206579e+01 -3.64794737e+01  2.50657895e-01
   4.00000000e+00]
 [ 1.55626429e+02  1.16517143e+01 -3.65504286e+01  7.62857143e-01
   3.00000000e+00]
 [ 2.62701842e+02  9.72447368e+00 -3.62463158e+01  3.46842105e-01
   2.00000000e+00]]


In [117]:
right = [0, 0, 0, 0, 0]
error = [0, 0, 0, 0, 0]
for i in range(1000):
    # 'x', 'y', 'xVelocity', 'yVelocity', 'state'
    current_data = [train_data2[i+2]]
    history_data1 = [train_data2[i]]
    history_data2 = [train_data2[i+1]]
    #print(history_data1,history_data2,current_data,)
    for j in range(5):        
        predicted_state = model2c.predict(current_data)
        #print('predict state is:',predicted_state)
        posterior_probs = model2c.predict_proba(current_data)
        most_probable_state = np.argmax(posterior_probs)
        predicted_observation = model2c.means_[most_probable_state]
        if (predicted_state == [3]):
            predict_result = current_data[0][4] 
        else:
            predict_result = predicted_observation[4]
        if(predict_result==int(train_data2[i+j+3][4])):
            right[j] = right[j] + 1
        else:
            error[j] = error[j] + 1
        
        next_vx = current_data[0][2] + ((current_data[0][2] - history_data2[0][2]) - (history_data2[0][2] - history_data1[0][2]))
        next_vy = current_data[0][3] + ((current_data[0][3] - history_data2[0][3]) - (history_data2[0][3] - history_data1[0][3]))
        next_x = 0.04 * 0.5 * (next_vx + current_data[0][2]) + current_data[0][0]
        next_y = 0.04 * 0.5 * (next_vy + current_data[0][3]) + current_data[0][1]

        current_data = [[next_x, next_y, next_vx, next_vy, predict_result]]
        history_data1 = history_data2
        history_data2 = current_data
        print('Predict result is: ', current_data)
        print('Real result is:', train_data2[i+j+3])

        
print('Right Num:', right, 'Error Num:', error)

Predict result is:  [[159.0774, 9.39, -32.059999999999995, 0.0, 2.0]]
Real result is: [159.13   9.39 -32.08   0.     2.  ]
Predict result is:  [[157.79500000000002, 9.39, -32.06, 0.0, 2.0]]
Real result is: [157.86   9.39 -32.09   0.     2.  ]
Predict result is:  [[156.51260000000002, 9.39, -32.059999999999995, 0.0, 2.0]]
Real result is: [156.59   9.39 -32.1    0.     2.  ]
Predict result is:  [[155.23020000000002, 9.39, -32.06, 0.0, 2.0]]
Real result is: [155.31   9.39 -32.11   0.     2.  ]
Predict result is:  [[153.94780000000003, 9.39, -32.059999999999995, 0.0, 2.0]]
Real result is: [154.03   9.39 -32.12   0.     2.  ]
Predict result is:  [[157.8468, 9.39, -32.08, 0.0, 2.0]]
Real result is: [157.86   9.39 -32.09   0.     2.  ]
Predict result is:  [[156.56380000000001, 9.39, -32.07, 0.0, 2.0]]
Real result is: [156.59   9.39 -32.1    0.     2.  ]
Predict result is:  [[155.28080000000003, 9.39, -32.08, 0.0, 2.0]]
Real result is: [155.31   9.39 -32.11   0.     2.  ]
Predict result is:  [

Predict result is:  [[82.76180000000001, 9.306600000000001, -32.46000000000001, -0.08, 2.0]]
Real result is: [ 82.76   9.31 -32.45  -0.09   2.  ]
Predict result is:  [[81.4638, 9.303200000000002, -32.44, -0.09, 2.0]]
Real result is: [ 81.47   9.3  -32.45  -0.09   2.  ]
Predict result is:  [[80.1658, 9.299800000000003, -32.46000000000001, -0.08, 2.0]]
Real result is: [ 80.17   9.3  -32.46  -0.09   2.  ]
Predict result is:  [[78.8678, 9.296400000000004, -32.44, -0.09, 2.0]]
Real result is: [ 78.89   9.3  -32.46  -0.09   2.  ]
Predict result is:  [[77.5698, 9.293000000000005, -32.46000000000001, -0.08, 2.0]]
Real result is: [ 77.59   9.29 -32.46  -0.09   2.  ]
Predict result is:  [[81.46220000000001, 9.3064, -32.44, -0.09, 2.0]]
Real result is: [ 81.47   9.3  -32.45  -0.09   2.  ]
Predict result is:  [[80.16440000000001, 9.3028, -32.45, -0.09, 2.0]]
Real result is: [ 80.17   9.3  -32.46  -0.09   2.  ]
Predict result is:  [[78.86660000000002, 9.299199999999999, -32.44, -0.09, 2.0]]
Real re

Predict result is:  [[-6.003199999999999, 9.197999999999999, -32.87, 0.04, 2.0]]
Real result is: [ 2.3905e+02  8.8500e+00 -3.1360e+01 -1.0000e-02  3.0000e+00]
Predict result is:  [[-2.0652000000000004, 9.1922, -32.88000000000001, 0.060000000000000005, 2.0]]
Real result is: [ -2.06   9.19 -32.89   0.05   2.  ]
Predict result is:  [[-3.3802000000000003, 9.1942, -32.87, 0.04, 2.0]]
Real result is: [ -3.38   9.19 -32.89   0.05   2.  ]
Predict result is:  [[-4.6952, 9.196200000000001, -32.88000000000001, 0.060000000000000005, 2.0]]
Real result is: [ -4.69   9.2  -32.9    0.05   2.  ]
Predict result is:  [[-6.010199999999999, 9.198200000000002, -32.87, 0.04, 2.0]]
Real result is: [ 2.3905e+02  8.8500e+00 -3.1360e+01 -1.0000e-02  3.0000e+00]
Predict result is:  [[-7.325199999999999, 9.200200000000002, -32.88000000000001, 0.060000000000000005, 2.0]]
Real result is: [ 2.3788e+02  8.8500e+00 -3.1360e+01 -1.0000e-02  3.0000e+00]
Predict result is:  [[-3.3756000000000004, 9.191799999999999, -32.88

Predict result is:  [[184.9992, 8.613399999999999, -31.52, -0.17, 3.0]]
Real result is: [ 1.850e+02  8.610e+00 -3.151e+01 -1.500e-01  3.000e+00]
Predict result is:  [[183.7384, 8.606799999999998, -31.52, -0.16, 3.0]]
Real result is: [ 1.8374e+02  8.6000e+00 -3.1510e+01 -1.5000e-01  3.0000e+00]
Predict result is:  [[182.47760000000002, 8.600199999999997, -31.52, -0.17, 3.0]]
Real result is: [ 1.8248e+02  8.6000e+00 -3.1510e+01 -1.4000e-01  3.0000e+00]
Predict result is:  [[181.21680000000003, 8.593599999999997, -31.52, -0.16, 3.0]]
Real result is: [ 1.8123e+02  8.5900e+00 -3.1510e+01 -1.3000e-01  3.0000e+00]
Predict result is:  [[179.95600000000005, 8.586999999999996, -31.52, -0.17, 3.0]]
Real result is: [ 1.7997e+02  8.5800e+00 -3.1510e+01 -1.3000e-01  3.0000e+00]
Predict result is:  [[183.7398, 8.604199999999999, -31.500000000000004, -0.13999999999999999, 3.0]]
Real result is: [ 1.8374e+02  8.6000e+00 -3.1510e+01 -1.5000e-01  3.0000e+00]
Predict result is:  [[182.4794, 8.5981999999999

Predict result is:  [[144.72099999999998, 8.5748, -31.4, 0.04, 3.0]]
Real result is: [ 1.4473e+02  8.5800e+00 -3.1400e+01  5.0000e-02  3.0000e+00]
Predict result is:  [[143.46459999999996, 8.5764, -31.42, 0.04, 3.0]]
Real result is: [ 1.4347e+02  8.5800e+00 -3.1390e+01  6.0000e-02  3.0000e+00]
Predict result is:  [[142.20819999999995, 8.578, -31.4, 0.04, 3.0]]
Real result is: [ 1.4222e+02  8.5800e+00 -3.1390e+01  6.0000e-02  3.0000e+00]
Predict result is:  [[145.9734, 8.5722, -31.42, 0.060000000000000005, 3.0]]
Real result is: [ 1.4598e+02  8.5700e+00 -3.1400e+01  5.0000e-02  3.0000e+00]
Predict result is:  [[144.7168, 8.574200000000001, -31.41, 0.04, 3.0]]
Real result is: [ 1.4473e+02  8.5800e+00 -3.1400e+01  5.0000e-02  3.0000e+00]
Predict result is:  [[143.46020000000001, 8.576200000000002, -31.42, 0.060000000000000005, 3.0]]
Real result is: [ 1.4347e+02  8.5800e+00 -3.1390e+01  6.0000e-02  3.0000e+00]
Predict result is:  [[142.20360000000002, 8.578200000000002, -31.41, 0.04, 3.0]]


...

Predict result is:  [[114.4098, 9.805199999999997, -37.95, -0.03, 2.0]]
Real result is: [ 1.144e+02  9.810e+00 -3.793e+01 -4.000e-02  2.000e+00]
Predict result is:  [[112.8922, 9.803999999999997, -37.92999999999999, -0.03, 2.0]]
Real result is: [ 1.1289e+02  9.8100e+00 -3.7930e+01 -4.0000e-02  2.0000e+00]
Predict result is:  [[117.4422, 9.8088, -37.95, -0.03, 2.0]]
Real result is: [ 1.1744e+02  9.8100e+00 -3.7940e+01 -4.0000e-02  2.0000e+00]
Predict result is:  [[115.9244, 9.807599999999999, -37.94, -0.03, 2.0]]
Real result is: [ 1.1592e+02  9.8100e+00 -3.7930e+01 -4.0000e-02  2.0000e+00]
Predict result is:  [[114.40660000000001, 9.806399999999998, -37.95, -0.03, 2.0]]
Real result is: [ 1.144e+02  9.810e+00 -3.793e+01 -4.000e-02  2.000e+00]
Predict result is:  [[112.88880000000002, 9.805199999999997, -37.94, -0.03, 2.0]]
Real result is: [ 1.1289e+02  9.8100e+00 -3.7930e+01 -4.0000e-02  2.0000e+00]
Predict result is:  [[111.37100000000002, 9.803999999999997, -37.95, -0.03, 2.0]]
Real re

Predict result is:  [[35.752, 9.623000000000001, -37.72, -0.1, 2.0]]
Real result is: [ 35.76   9.62 -37.71  -0.08   2.  ]
Predict result is:  [[34.243, 9.6194, -37.73, -0.07999999999999999, 2.0]]
Real result is: [ 34.25   9.62 -37.7   -0.08   2.  ]
Predict result is:  [[32.734, 9.6158, -37.72, -0.1, 2.0]]
Real result is: [ 32.74   9.61 -37.7   -0.07   2.  ]
Predict result is:  [[31.225, 9.6122, -37.73, -0.07999999999999999, 2.0]]
Real result is: [ 31.22   9.61 -37.69  -0.07   2.  ]
Predict result is:  [[35.751799999999996, 9.6262, -37.7, -0.1, 2.0]]
Real result is: [ 35.76   9.62 -37.71  -0.08   2.  ]
Predict result is:  [[34.243399999999994, 9.6224, -37.72, -0.09, 2.0]]
Real result is: [ 34.25   9.62 -37.7   -0.08   2.  ]
Predict result is:  [[32.73499999999999, 9.6186, -37.7, -0.1, 2.0]]
Real result is: [ 32.74   9.61 -37.7   -0.07   2.  ]
Predict result is:  [[31.22659999999999, 9.6148, -37.72, -0.09, 2.0]]
Real result is: [ 31.22   9.61 -37.69  -0.07   2.  ]
Predict result is:  [[2

Predict result is:  [[340.58799999999997, 8.879000000000003, -31.51, 0.14, 3.0]]
Real result is: [ 3.4059e+02  8.8800e+00 -3.1530e+01  1.6000e-01  3.0000e+00]
Predict result is:  [[344.36899999999997, 8.856, -31.529999999999998, 0.15, 3.0]]
Real result is: [ 3.4437e+02  8.8600e+00 -3.1520e+01  1.6000e-01  3.0000e+00]
Predict result is:  [[343.10819999999995, 8.862, -31.51, 0.15, 3.0]]
Real result is: [ 3.4312e+02  8.8700e+00 -3.1520e+01  1.6000e-01  3.0000e+00]
Predict result is:  [[341.84739999999994, 8.868, -31.529999999999998, 0.15, 3.0]]
Real result is: [ 3.4184e+02  8.8700e+00 -3.1520e+01  1.6000e-01  3.0000e+00]
Predict result is:  [[340.5865999999999, 8.874, -31.51, 0.15, 3.0]]
Real result is: [ 3.4059e+02  8.8800e+00 -3.1530e+01  1.6000e-01  3.0000e+00]
Predict result is:  [[339.3257999999999, 8.88, -31.529999999999998, 0.15, 3.0]]
Real result is: [ 3.3934e+02  8.8900e+00 -3.1530e+01  1.7000e-01  3.0000e+00]
Predict result is:  [[343.1094, 8.8666, -31.51, 0.17, 3.0]]
Real resul

Predict result is:  [[291.3984, 9.178799999999997, -31.51, 0.18, 2.0]]
Real result is: [ 2.9141e+02  9.1800e+00 -3.1500e+01  1.7000e-01  2.0000e+00]
Predict result is:  [[290.138, 9.185999999999996, -31.51, 0.18, 2.0]]
Real result is: [ 2.9014e+02  9.1800e+00 -3.1490e+01  1.7000e-01  2.0000e+00]
Predict result is:  [[293.9202, 9.1666, -31.49, 0.16000000000000003, 2.0]]
Real result is: [ 2.9392e+02  9.1600e+00 -3.1500e+01  1.7000e-01  2.0000e+00]
Predict result is:  [[292.66020000000003, 9.1734, -31.51, 0.18, 2.0]]
Real result is: [ 2.9267e+02  9.1700e+00 -3.1500e+01  1.7000e-01  2.0000e+00]
Predict result is:  [[291.40020000000004, 9.180200000000001, -31.49, 0.16000000000000003, 2.0]]
Real result is: [ 2.9141e+02  9.1800e+00 -3.1500e+01  1.7000e-01  2.0000e+00]
Predict result is:  [[290.14020000000005, 9.187000000000001, -31.51, 0.18, 2.0]]
Real result is: [ 2.9014e+02  9.1800e+00 -3.1490e+01  1.7000e-01  2.0000e+00]
Predict result is:  [[288.88020000000006, 9.193800000000001, -31.49, 

Predict result is:  [[219.8012, 9.594999999999999, -31.289999999999996, 0.18, 2.0]]
Real result is: [ 2.1981e+02  9.6000e+00 -3.1260e+01  1.4000e-01  2.0000e+00]
Predict result is:  [[223.5582, 9.5762, -31.3, 0.15, 2.0]]
Real result is: [ 2.2357e+02  9.5800e+00 -3.1280e+01  1.6000e-01  2.0000e+00]
Predict result is:  [[222.3064, 9.5826, -31.29, 0.17, 2.0]]
Real result is: [ 2.2232e+02  9.5800e+00 -3.1270e+01  1.5000e-01  2.0000e+00]
Predict result is:  [[221.0546, 9.588999999999999, -31.3, 0.15, 2.0]]
Real result is: [ 2.2107e+02  9.5900e+00 -3.1270e+01  1.4000e-01  2.0000e+00]
Predict result is:  [[219.8028, 9.595399999999998, -31.29, 0.17, 2.0]]
Real result is: [ 2.1981e+02  9.6000e+00 -3.1260e+01  1.4000e-01  2.0000e+00]
Predict result is:  [[218.551, 9.601799999999997, -31.3, 0.15, 2.0]]
Real result is: [ 2.1856e+02  9.6000e+00 -3.1250e+01  1.3000e-01  2.0000e+00]
Predict result is:  [[222.319, 9.5866, -31.270000000000003, 0.17, 2.0]]
Real result is: [ 2.2232e+02  9.5800e+00 -3.127

In [120]:
df = pd.read_csv('train_dataset6c.csv')

train_data6c = df[['x', 'y', 'xVelocity', 'yVelocity', 'state']].values
print(train_data6c)
model6c = hmm.GaussianHMM(n_components=4, covariance_type="full", n_iter=10000)

model6c.fit(train_data6c)

print("Training Finish!")
print(model6c.means_)


[[ 1.8150e+02  2.5490e+01  2.5070e+01  9.0000e-02  7.0000e+00]
 [ 1.8239e+02  2.5490e+01  2.5060e+01  9.0000e-02  7.0000e+00]
 [ 1.8333e+02  2.5490e+01  2.5050e+01  8.0000e-02  7.0000e+00]
 ...
 [ 4.1111e+02  2.1880e+01  2.6840e+01 -1.2000e-01  5.0000e+00]
 [ 4.1219e+02  2.1870e+01  2.6870e+01 -1.0000e-01  5.0000e+00]
 [ 4.1326e+02  2.1870e+01  2.6900e+01 -9.0000e-02  5.0000e+00]]
Training Finish!
[[ 2.88337536e+02  2.53076038e+01  2.39363683e+01 -1.72834342e-01
   7.00000000e+00]
 [ 3.87899592e+02  2.22387755e+01  2.61804082e+01 -5.54897959e-01
   5.00000000e+00]
 [ 2.21480092e+02  2.54333894e+01  2.45821942e+01 -3.76676051e-02
   7.00000000e+00]
 [ 3.38506458e+02  2.40229167e+01  2.47018750e+01 -1.05020833e+00
   6.00000000e+00]]


In [121]:
right = [0, 0, 0, 0, 0]
error = [0, 0, 0, 0, 0]
for i in range(1000):
    # 'x', 'y', 'xVelocity', 'yVelocity', 'state'
    current_data = [train_data3[i+2]]
    history_data1 = [train_data3[i]]
    history_data2 = [train_data3[i+1]]
    #print(history_data1,history_data2,current_data,)
    for j in range(5):        
        predicted_state = model2c.predict(current_data)
        #print('predict state is:',predicted_state)
        posterior_probs = model2c.predict_proba(current_data)
        most_probable_state = np.argmax(posterior_probs)
        predicted_observation = model2c.means_[most_probable_state]
        if (predicted_state == [3]):
            predict_result = current_data[0][4] 
        else:
            predict_result = predicted_observation[4]
        if(predict_result==int(train_data3[i+j+3][4])):
            right[j] = right[j] + 1
        else:
            error[j] = error[j] + 1
        
        next_vx = current_data[0][2] + ((current_data[0][2] - history_data2[0][2]) - (history_data2[0][2] - history_data1[0][2]))
        next_vy = current_data[0][3] + ((current_data[0][3] - history_data2[0][3]) - (history_data2[0][3] - history_data1[0][3]))
        next_x = 0.04 * 0.5 * (next_vx + current_data[0][2]) + current_data[0][0]
        next_y = 0.04 * 0.5 * (next_vy + current_data[0][3]) + current_data[0][1]

        current_data = [[next_x, next_y, next_vx, next_vy, predict_result]]
        history_data1 = history_data2
        history_data2 = current_data
        print('Predict result is: ', current_data)
        print('Real result is:', train_data3[i+j+3])

        
print('Right Num:', right, 'Error Num:', error)

Predict result is:  [[262.431, 13.134200000000002, -42.72, 0.11000000000000001, 4.0]]
Real result is: [ 2.6248e+02  1.3130e+01 -4.2730e+01  1.0000e-01  4.0000e+00]
Predict result is:  [[260.722, 13.138200000000001, -42.73, 0.09, 4.0]]
Real result is: [ 2.608e+02  1.314e+01 -4.274e+01  1.000e-01  4.000e+00]
Predict result is:  [[259.013, 13.1422, -42.72, 0.11000000000000001, 4.0]]
Real result is: [ 2.591e+02  1.314e+01 -4.274e+01  1.000e-01  4.000e+00]
Predict result is:  [[257.304, 13.1462, -42.73, 0.09, 4.0]]
Real result is: [ 2.574e+02  1.315e+01 -4.275e+01  1.000e-01  4.000e+00]
Predict result is:  [[255.59499999999997, 13.1502, -42.72, 0.11000000000000001, 4.0]]
Real result is: [ 2.5569e+02  1.3150e+01 -4.2750e+01  1.1000e-01  4.0000e+00]
Predict result is:  [[260.7708, 13.1338, -42.73, 0.09, 4.0]]
Real result is: [ 2.608e+02  1.314e+01 -4.274e+01  1.000e-01  4.000e+00]
Predict result is:  [[259.0616, 13.1376, -42.73, 0.1, 4.0]]
Real result is: [ 2.591e+02  1.314e+01 -4.274e+01  1.

Predict result is:  [[204.16000000000005, 13.302000000000001, -42.8, 0.11, 4.0]]
Real result is: [ 2.0417e+02  1.3300e+01 -4.2790e+01  9.0000e-02  4.0000e+00]
Predict result is:  [[209.2986, 13.2844, -42.78, 0.11, 4.0]]
Real result is: [ 2.093e+02  1.329e+01 -4.279e+01  1.000e-01  4.000e+00]
Predict result is:  [[207.587, 13.2888, -42.8, 0.11, 4.0]]
Real result is: [ 2.0759e+02  1.3290e+01 -4.2790e+01  1.0000e-01  4.0000e+00]
Predict result is:  [[205.87539999999998, 13.2932, -42.78, 0.11, 4.0]]
Real result is: [ 2.0588e+02  1.3300e+01 -4.2790e+01  1.0000e-01  4.0000e+00]
Predict result is:  [[204.16379999999998, 13.297600000000001, -42.8, 0.11, 4.0]]
Real result is: [ 2.0417e+02  1.3300e+01 -4.2790e+01  9.0000e-02  4.0000e+00]
Predict result is:  [[202.45219999999998, 13.302000000000001, -42.78, 0.11, 4.0]]
Real result is: [ 2.0246e+02  1.3300e+01 -4.2790e+01  9.0000e-02  4.0000e+00]
Predict result is:  [[207.5882, 13.2938, -42.8, 0.09000000000000001, 4.0]]
Real result is: [ 2.0759e+0

Predict result is:  [[144.3796, 13.296600000000002, -42.76, -0.08, 4.0]]
Real result is: [ 1.4438e+02  1.3290e+01 -4.2760e+01 -9.0000e-02  4.0000e+00]
Predict result is:  [[142.66920000000002, 13.293200000000002, -42.76, -0.09, 4.0]]
Real result is: [ 1.4268e+02  1.3290e+01 -4.2770e+01 -1.0000e-01  4.0000e+00]
Predict result is:  [[140.95880000000002, 13.289800000000003, -42.76, -0.08, 4.0]]
Real result is: [ 1.4097e+02  1.3290e+01 -4.2770e+01 -1.0000e-01  4.0000e+00]
Predict result is:  [[139.24840000000003, 13.286400000000004, -42.76, -0.09, 4.0]]
Real result is: [ 1.3926e+02  1.3280e+01 -4.2770e+01 -1.0000e-01  4.0000e+00]
Predict result is:  [[137.53800000000004, 13.283000000000005, -42.76, -0.08, 4.0]]
Real result is: [ 1.3754e+02  1.3280e+01 -4.2770e+01 -1.1000e-01  4.0000e+00]
Predict result is:  [[142.6696, 13.286399999999999, -42.76, -0.09, 4.0]]
Real result is: [ 1.4268e+02  1.3290e+01 -4.2770e+01 -1.0000e-01  4.0000e+00]
Predict result is:  [[140.9592, 13.282799999999998, -4

Predict result is:  [[10.844999999999995, 13.079, -42.68, 0.09, 4.0]]
Real result is: [ 10.9   13.09 -42.63   0.14   4.  ]
Predict result is:  [[15.963800000000003, 13.0746, -42.64999999999999, 0.12, 4.0]]
Real result is: [ 15.97  13.07 -42.65   0.12   4.  ]
Predict result is:  [[14.257400000000002, 13.079, -42.67, 0.1, 4.0]]
Real result is: [ 14.28  13.08 -42.65   0.13   4.  ]
Predict result is:  [[12.551000000000002, 13.083400000000001, -42.64999999999999, 0.12, 4.0]]
Real result is: [ 12.59  13.08 -42.64   0.13   4.  ]
Predict result is:  [[10.844600000000002, 13.087800000000001, -42.67, 0.1, 4.0]]
Real result is: [ 10.9   13.09 -42.63   0.14   4.  ]
Predict result is:  [[9.138200000000001, 13.092200000000002, -42.64999999999999, 0.12, 4.0]]
Real result is: [  9.19  13.1  -42.63   0.15   4.  ]
Predict result is:  [[14.264000000000001, 13.0748, -42.650000000000006, 0.12, 4.0]]
Real result is: [ 14.28  13.08 -42.65   0.13   4.  ]
Predict result is:  [[12.5578, 13.0794, -42.66, 0.11, 4

Predict result is:  [[261.8632, 13.7668, -42.94, -0.04, 4.0]]
Real result is: [ 2.6186e+02  1.3770e+01 -4.2990e+01 -4.0000e-02  4.0000e+00]
Predict result is:  [[260.145, 13.7652, -42.970000000000006, -0.04, 4.0]]
Real result is: [ 2.6014e+02  1.3770e+01 -4.3010e+01 -3.0000e-02  4.0000e+00]
Predict result is:  [[258.42679999999996, 13.7636, -42.94, -0.04, 4.0]]
Real result is: [ 2.5842e+02  1.3770e+01 -4.3030e+01 -3.0000e-02  4.0000e+00]
Predict result is:  [[256.70859999999993, 13.762, -42.970000000000006, -0.04, 4.0]]
Real result is: [ 2.567e+02  1.377e+01 -4.304e+01 -3.000e-02  4.000e+00]
Predict result is:  [[261.8608, 13.7684, -42.97999999999999, -0.04, 4.0]]
Real result is: [ 2.6186e+02  1.3770e+01 -4.2990e+01 -4.0000e-02  4.0000e+00]
Predict result is:  [[260.142, 13.7668, -42.96, -0.04, 4.0]]
Real result is: [ 2.6014e+02  1.3770e+01 -4.3010e+01 -3.0000e-02  4.0000e+00]
Predict result is:  [[258.4232, 13.7652, -42.97999999999999, -0.04, 4.0]]
Real result is: [ 2.5842e+02  1.3770

...

Predict result is:  [[140.9294, 13.1552, -31.009999999999998, -0.12, 4.0]]
Real result is: [ 1.4092e+02  1.3150e+01 -3.1010e+01 -1.2000e-01  4.0000e+00]
Predict result is:  [[139.68859999999998, 13.150400000000001, -31.03, -0.12, 4.0]]
Real result is: [ 1.3968e+02  1.3150e+01 -3.1010e+01 -1.2000e-01  4.0000e+00]
Predict result is:  [[138.44779999999997, 13.145600000000002, -31.009999999999998, -0.12, 4.0]]
Real result is: [ 1.3844e+02  1.3140e+01 -3.1000e+01 -1.2000e-01  4.0000e+00]
Predict result is:  [[137.20699999999997, 13.140800000000002, -31.03, -0.12, 4.0]]
Real result is: [ 1.372e+02  1.314e+01 -3.100e+01 -1.300e-01  4.000e+00]
Predict result is:  [[135.96619999999996, 13.136000000000003, -31.009999999999998, -0.12, 4.0]]
Real result is: [ 1.3596e+02  1.3140e+01 -3.0990e+01 -1.3000e-01  4.0000e+00]
Predict result is:  [[139.6796, 13.1452, -31.010000000000005, -0.12, 4.0]]
Real result is: [ 1.3968e+02  1.3150e+01 -3.1010e+01 -1.2000e-01  4.0000e+00]
Predict result is:  [[138.439

Predict result is:  [[84.099, 12.676200000000001, -30.83, -0.46, 3.0]]
Real result is: [ 84.12  12.67 -30.82  -0.52   3.  ]
Predict result is:  [[82.8662, 12.657800000000002, -30.810000000000002, -0.4599999999999999, 3.0]]
Real result is: [ 82.89  12.65 -30.82  -0.53   3.  ]
Predict result is:  [[86.567, 12.7208, -30.83, -0.48000000000000004, 3.0]]
Real result is: [ 86.57  12.72 -30.82  -0.49   3.  ]
Predict result is:  [[85.33399999999999, 12.7018, -30.82, -0.47, 3.0]]
Real result is: [ 85.35  12.7  -30.82  -0.51   3.  ]
Predict result is:  [[84.10099999999998, 12.6828, -30.83, -0.48000000000000004, 3.0]]
Real result is: [ 84.12  12.67 -30.82  -0.52   3.  ]
Predict result is:  [[82.86799999999998, 12.6638, -30.82, -0.47, 3.0]]
Real result is: [ 82.89  12.65 -30.82  -0.53   3.  ]
Predict result is:  [[81.63499999999998, 12.6448, -30.83, -0.48000000000000004, 3.0]]
Real result is: [ 81.66  12.63 -30.81  -0.54   3.  ]
Predict result is:  [[85.3372, 12.7004, -30.82, -0.49, 3.0]]
Real resu

Predict result is:  [[2.7927999999999997, 10.3478, -30.81, -1.11, 3.0]]
Real result is: [  2.77  10.35 -30.81  -1.12   3.  ]
Predict result is:  [[1.5603999999999998, 10.303799999999999, -30.81, -1.09, 3.0]]
Real result is: [ 4.0447e+02  1.2260e+01 -3.4650e+01  3.6000e-01  3.0000e+00]
Predict result is:  [[0.32799999999999985, 10.259799999999998, -30.81, -1.11, 3.0]]
Real result is: [ 4.0313e+02  1.2280e+01 -3.4650e+01  3.6000e-01  3.0000e+00]
Predict result is:  [[4.0276, 10.3962, -30.81, -1.09, 3.0]]
Real result is: [  4.02  10.39 -30.81  -1.11   3.  ]
Predict result is:  [[2.7951999999999995, 10.352400000000001, -30.81, -1.1, 3.0]]
Real result is: [  2.77  10.35 -30.81  -1.12   3.  ]
Predict result is:  [[1.5627999999999995, 10.308600000000002, -30.81, -1.09, 3.0]]
Real result is: [ 4.0447e+02  1.2260e+01 -3.4650e+01  3.6000e-01  3.0000e+00]
Predict result is:  [[0.3303999999999996, 10.264800000000003, -30.81, -1.1, 3.0]]
Real result is: [ 4.0313e+02  1.2280e+01 -3.4650e+01  3.6000e

Predict result is:  [[325.62459999999993, 12.894800000000002, -34.68, 0.18, 4.0]]
Real result is: [ 3.2564e+02  1.2900e+01 -3.4690e+01  1.7000e-01  4.0000e+00]
Predict result is:  [[324.2369999999999, 12.902200000000002, -34.699999999999996, 0.19, 4.0]]
Real result is: [ 3.2426e+02  1.2900e+01 -3.4690e+01  1.6000e-01  4.0000e+00]
Predict result is:  [[322.8493999999999, 12.909600000000003, -34.68, 0.18, 4.0]]
Real result is: [ 3.2289e+02  1.2910e+01 -3.4690e+01  1.6000e-01  4.0000e+00]
Predict result is:  [[321.46179999999987, 12.917000000000003, -34.699999999999996, 0.19, 4.0]]
Real result is: [ 3.2151e+02  1.2910e+01 -3.4690e+01  1.5000e-01  4.0000e+00]
Predict result is:  [[325.6226, 12.896600000000001, -34.68, 0.16000000000000003, 4.0]]
Real result is: [ 3.2564e+02  1.2900e+01 -3.4690e+01  1.7000e-01  4.0000e+00]
Predict result is:  [[324.23519999999996, 12.903400000000001, -34.69, 0.18, 4.0]]
Real result is: [ 3.2426e+02  1.2900e+01 -3.4690e+01  1.6000e-01  4.0000e+00]
Predict res

Predict result is:  [[279.81080000000003, 13.036399999999999, -34.74, 0.08, 4.0]]
Real result is: [ 2.7982e+02  1.3030e+01 -3.4740e+01  8.0000e-02  4.0000e+00]
Predict result is:  [[278.42120000000006, 13.039599999999998, -34.74, 0.08, 4.0]]
Real result is: [ 2.7842e+02  1.3040e+01 -3.4740e+01  8.0000e-02  4.0000e+00]
Predict result is:  [[277.0316000000001, 13.042799999999998, -34.74, 0.08, 4.0]]
Real result is: [ 2.7702e+02  1.3040e+01 -3.4740e+01  9.0000e-02  4.0000e+00]
Predict result is:  [[275.6420000000001, 13.045999999999998, -34.74, 0.08, 4.0]]
Real result is: [ 2.7563e+02  1.3040e+01 -3.4740e+01  9.0000e-02  4.0000e+00]
Predict result is:  [[279.8204, 13.033199999999999, -34.74, 0.08, 4.0]]
Real result is: [ 2.7982e+02  1.3030e+01 -3.4740e+01  8.0000e-02  4.0000e+00]
Predict result is:  [[278.43080000000003, 13.036399999999999, -34.74, 0.08, 4.0]]
Real result is: [ 2.7842e+02  1.3040e+01 -3.4740e+01  8.0000e-02  4.0000e+00]
Predict result is:  [[277.04120000000006, 13.0395999

In [125]:
right = [0, 0, 0, 0, 0]
error = [0, 0, 0, 0, 0]
for i in range(1000):
    # 'x', 'y', 'xVelocity', 'yVelocity', 'state'
    current_data = [train_data5[i+2]]
    history_data1 = [train_data5[i]]
    history_data2 = [train_data5[i+1]]
    #print(history_data1,history_data2,current_data,)
    for j in range(5):        
        predicted_state = model6c.predict(current_data)
        #print('predict state is:',predicted_state)
        posterior_probs = model6c.predict_proba(current_data)
        most_probable_state = np.argmax(posterior_probs)
        predicted_observation = model6c.means_[most_probable_state]
        if (predicted_state == [2]):
            predict_result = current_data[0][4] 
        else:
            predict_result = predicted_observation[4]
        if(predict_result==int(train_data5[i+j+3][4])):
            right[j] = right[j] + 1
        else:
            error[j] = error[j] + 1
        
        next_vx = current_data[0][2] + ((current_data[0][2] - history_data2[0][2]) - (history_data2[0][2] - history_data1[0][2]))
        next_vy = current_data[0][3] + ((current_data[0][3] - history_data2[0][3]) - (history_data2[0][3] - history_data1[0][3]))
        next_x = 0.04 * 0.5 * (next_vx + current_data[0][2]) + current_data[0][0]
        next_y = 0.04 * 0.5 * (next_vy + current_data[0][3]) + current_data[0][1]

        current_data = [[next_x, next_y, next_vx, next_vy, predict_result]]
        history_data1 = history_data2
        history_data2 = current_data
        print('Predict result is: ', current_data)
        print('Real result is:', train_data5[i+j+3])

        
print('Right Num:', right, 'Error Num:', error)

Predict result is:  [[366.905, 21.68, 40.87000000000001, 0.0, 5.0]]
Real result is: [366.83  21.68  40.89   0.     5.  ]
Predict result is:  [[368.53979999999996, 21.68, 40.87, 0.0, 5.0]]
Real result is: [3.6842e+02 2.1680e+01 4.0900e+01 1.0000e-02 5.0000e+00]
Predict result is:  [[370.17459999999994, 21.68, 40.87000000000001, 0.0, 5.0]]
Real result is: [3.7004e+02 2.1680e+01 4.0920e+01 1.0000e-02 5.0000e+00]
Predict result is:  [[371.8093999999999, 21.68, 40.87, 0.0, 5.0]]
Real result is: [3.7168e+02 2.1690e+01 4.0930e+01 1.0000e-02 5.0000e+00]
Predict result is:  [[373.4441999999999, 21.68, 40.87000000000001, 0.0, 5.0]]
Real result is: [3.7332e+02 2.1690e+01 4.0940e+01 1.0000e-02 5.0000e+00]
Predict result is:  [[368.4656, 21.68, 40.88999999999999, 0.0, 5.0]]
Real result is: [3.6842e+02 2.1680e+01 4.0900e+01 1.0000e-02 5.0000e+00]
Predict result is:  [[370.101, 21.68, 40.88, 0.0, 5.0]]
Real result is: [3.7004e+02 2.1680e+01 4.0920e+01 1.0000e-02 5.0000e+00]
Predict result is:  [[371.

Predict result is:  [[193.21959999999999, 21.6396, 35.74000000000001, -0.01, 5.0]]
Real result is: [ 1.9321e+02  2.1640e+01  3.5750e+01 -1.0000e-02  5.0000e+00]
Predict result is:  [[194.64899999999997, 21.639200000000002, 35.73, -0.01, 5.0]]
Real result is: [ 1.9464e+02  2.1640e+01  3.5750e+01 -1.0000e-02  5.0000e+00]
Predict result is:  [[196.07839999999996, 21.638800000000003, 35.74000000000001, -0.01, 5.0]]
Real result is: [ 1.9607e+02  2.1640e+01  3.5760e+01 -1.0000e-02  5.0000e+00]
Predict result is:  [[197.50779999999995, 21.638400000000004, 35.73, -0.01, 5.0]]
Real result is: [ 1.9751e+02  2.1640e+01  3.5770e+01 -1.0000e-02  5.0000e+00]
Predict result is:  [[198.93719999999993, 21.638000000000005, 35.74000000000001, -0.01, 5.0]]
Real result is: [ 1.9894e+02  2.1640e+01  3.5780e+01 -2.0000e-02  5.0000e+00]
Predict result is:  [[194.64000000000001, 21.6396, 35.74999999999999, -0.01, 5.0]]
Real result is: [ 1.9464e+02  2.1640e+01  3.5750e+01 -1.0000e-02  5.0000e+00]
Predict result

Predict result is:  [[282.50939999999997, 21.5536, 36.11, -0.04, 5.0]]
Real result is: [ 2.8249e+02  2.1550e+01  3.6130e+01 -4.0000e-02  5.0000e+00]
Predict result is:  [[283.95419999999996, 21.552, 36.129999999999995, -0.04, 5.0]]
Real result is: [ 2.8393e+02  2.1550e+01  3.6130e+01 -4.0000e-02  5.0000e+00]
Predict result is:  [[279.6146, 21.5584, 36.11, -0.04, 5.0]]
Real result is: [ 2.7962e+02  2.1560e+01  3.6120e+01 -4.0000e-02  5.0000e+00]
Predict result is:  [[281.0592, 21.5568, 36.12, -0.04, 5.0]]
Real result is: [ 2.8106e+02  2.1550e+01  3.6130e+01 -4.0000e-02  5.0000e+00]
Predict result is:  [[282.50379999999996, 21.5552, 36.11, -0.04, 5.0]]
Real result is: [ 2.8249e+02  2.1550e+01  3.6130e+01 -4.0000e-02  5.0000e+00]
Predict result is:  [[283.94839999999994, 21.5536, 36.12, -0.04, 5.0]]
Real result is: [ 2.8393e+02  2.1550e+01  3.6130e+01 -4.0000e-02  5.0000e+00]
Predict result is:  [[285.3929999999999, 21.552, 36.11, -0.04, 5.0]]
Real result is: [ 2.8538e+02  2.1550e+01  3.6

Predict result is:  [[325.9186, 21.5194, 36.21, -0.010000000000000002, 5.0]]
Real result is: [ 3.2592e+02  2.1520e+01  3.6220e+01 -2.0000e-02  5.0000e+00]
Predict result is:  [[327.3672, 21.5186, 36.22, -0.03, 5.0]]
Real result is: [ 3.2736e+02  2.1520e+01  3.6230e+01 -2.0000e-02  5.0000e+00]
Predict result is:  [[328.8158, 21.517799999999998, 36.21, -0.010000000000000002, 5.0]]
Real result is: [ 3.2881e+02  2.1520e+01  3.6230e+01 -2.0000e-02  5.0000e+00]
Predict result is:  [[330.2644, 21.516999999999996, 36.22, -0.03, 5.0]]
Real result is: [ 3.3026e+02  2.1520e+01  3.6230e+01 -2.0000e-02  5.0000e+00]
Predict result is:  [[331.713, 21.516199999999994, 36.21, -0.010000000000000002, 5.0]]
Real result is: [ 3.3171e+02  2.1520e+01  3.6240e+01 -2.0000e-02  5.0000e+00]
Predict result is:  [[327.3688, 21.519, 36.22, -0.03, 5.0]]
Real result is: [ 3.2736e+02  2.1520e+01  3.6230e+01 -2.0000e-02  5.0000e+00]
Predict result is:  [[328.8176, 21.517999999999997, 36.22, -0.02, 5.0]]
Real result is:

Predict result is:  [[190.54900000000004, 21.985200000000003, 32.93, -0.15, 5.0]]
Real result is: [ 1.9058e+02  2.1990e+01  3.3000e+01 -1.4000e-01  5.0000e+00]
Predict result is:  [[191.86620000000005, 21.979000000000003, 32.92999999999999, -0.16, 5.0]]
Real result is: [ 1.919e+02  2.199e+01  3.302e+01 -1.400e-01  5.000e+00]
Predict result is:  [[187.9086, 21.994, 32.970000000000006, -0.15, 5.0]]
Real result is: [ 1.8792e+02  2.2000e+01  3.2970e+01 -1.4000e-01  5.0000e+00]
Predict result is:  [[189.2268, 21.988, 32.94, -0.15, 5.0]]
Real result is: [ 1.8925e+02  2.1990e+01  3.2990e+01 -1.4000e-01  5.0000e+00]
Predict result is:  [[190.545, 21.982, 32.970000000000006, -0.15, 5.0]]
Real result is: [ 1.9058e+02  2.1990e+01  3.3000e+01 -1.4000e-01  5.0000e+00]
Predict result is:  [[191.86319999999998, 21.976, 32.94, -0.15, 5.0]]
Real result is: [ 1.919e+02  2.199e+01  3.302e+01 -1.400e-01  5.000e+00]
Predict result is:  [[193.18139999999997, 21.97, 32.970000000000006, -0.15, 5.0]]
Real resu

...

Predict result is:  [[384.7491999999999, 22.0844, 42.50000000000001, 0.12, 5.0]]
Real result is: [3.8475e+02 2.2090e+01 4.2560e+01 1.2000e-01 5.0000e+00]
Predict result is:  [[386.4487999999999, 22.089199999999998, 42.48, 0.12, 5.0]]
Real result is: [3.8645e+02 2.2090e+01 4.2580e+01 1.2000e-01 5.0000e+00]
Predict result is:  [[388.14839999999987, 22.093999999999998, 42.50000000000001, 0.12, 5.0]]
Real result is: [3.8817e+02 2.2100e+01 4.2600e+01 1.2000e-01 5.0000e+00]
Predict result is:  [[383.05080000000004, 22.084799999999998, 42.52, 0.12, 5.0]]
Real result is: [3.8304e+02 2.2080e+01 4.2540e+01 1.2000e-01 5.0000e+00]
Predict result is:  [[384.75120000000004, 22.089599999999997, 42.5, 0.12, 5.0]]
Real result is: [3.8475e+02 2.2090e+01 4.2560e+01 1.2000e-01 5.0000e+00]
Predict result is:  [[386.45160000000004, 22.094399999999997, 42.52, 0.12, 5.0]]
Real result is: [3.8645e+02 2.2090e+01 4.2580e+01 1.2000e-01 5.0000e+00]
Predict result is:  [[388.15200000000004, 22.099199999999996, 42.5

Predict result is:  [[82.53200000000001, 22.06200000000001, 35.81, 0.06, 5.0]]
Real result is: [8.253e+01 2.206e+01 3.582e+01 7.000e-02 5.000e+00]
Predict result is:  [[78.2424, 22.052400000000002, 35.81, 0.06, 5.0]]
Real result is: [7.824e+01 2.205e+01 3.582e+01 6.000e-02 5.000e+00]
Predict result is:  [[79.6748, 22.054800000000004, 35.81, 0.06, 5.0]]
Real result is: [7.967e+01 2.205e+01 3.582e+01 6.000e-02 5.000e+00]
Predict result is:  [[81.1072, 22.057200000000005, 35.81, 0.06, 5.0]]
Real result is: [8.110e+01 2.205e+01 3.582e+01 6.000e-02 5.000e+00]
Predict result is:  [[82.53960000000001, 22.059600000000007, 35.81, 0.06, 5.0]]
Real result is: [8.253e+01 2.206e+01 3.582e+01 7.000e-02 5.000e+00]
Predict result is:  [[83.97200000000001, 22.06200000000001, 35.81, 0.06, 5.0]]
Real result is: [8.396e+01 2.206e+01 3.582e+01 7.000e-02 5.000e+00]
Predict result is:  [[79.673, 22.052400000000002, 35.83, 0.06, 5.0]]
Real result is: [7.967e+01 2.205e+01 3.582e+01 6.000e-02 5.000e+00]
Predict

Predict result is:  [[192.9606, 22.288800000000002, 35.77, -0.02, 5.0]]
Real result is: [ 1.9297e+02  2.2290e+01  3.5750e+01 -3.0000e-02  5.0000e+00]
Predict result is:  [[194.391, 22.288200000000003, 35.74999999999999, -0.01, 5.0]]
Real result is: [ 1.944e+02  2.229e+01  3.574e+01 -4.000e-02  5.000e+00]
Predict result is:  [[195.82139999999998, 22.287600000000005, 35.77, -0.02, 5.0]]
Real result is: [ 1.9583e+02  2.2290e+01  3.5740e+01 -4.0000e-02  5.0000e+00]
Predict result is:  [[197.25179999999997, 22.287000000000006, 35.74999999999999, -0.01, 5.0]]
Real result is: [ 1.9725e+02  2.2280e+01  3.5730e+01 -5.0000e-02  5.0000e+00]
Predict result is:  [[192.9706, 22.2886, 35.77, -0.039999999999999994, 5.0]]
Real result is: [ 1.9297e+02  2.2290e+01  3.5750e+01 -3.0000e-02  5.0000e+00]
Predict result is:  [[194.4012, 22.287399999999998, 35.76, -0.02, 5.0]]
Real result is: [ 1.944e+02  2.229e+01  3.574e+01 -4.000e-02  5.000e+00]
Predict result is:  [[195.8318, 22.286199999999997, 35.77, -0.

Predict result is:  [[257.0182, 22.036, 35.49, -0.2, 5.0]]
Real result is: [ 2.5702e+02  2.2030e+01  3.5460e+01 -2.0000e-01  5.0000e+00]
Predict result is:  [[258.4376, 22.028000000000002, 35.48, -0.2, 5.0]]
Real result is: [ 2.5844e+02  2.2030e+01  3.5450e+01 -2.0000e-01  5.0000e+00]
Predict result is:  [[259.85699999999997, 22.020000000000003, 35.49, -0.2, 5.0]]
Real result is: [ 2.5987e+02  2.2020e+01  3.5440e+01 -1.9000e-01  5.0000e+00]
Predict result is:  [[255.5986, 22.042, 35.46, -0.2, 5.0]]
Real result is: [ 2.556e+02  2.204e+01  3.546e+01 -2.000e-01  5.000e+00]
Predict result is:  [[257.0174, 22.034000000000002, 35.48, -0.2, 5.0]]
Real result is: [ 2.5702e+02  2.2030e+01  3.5460e+01 -2.0000e-01  5.0000e+00]
Predict result is:  [[258.4362, 22.026000000000003, 35.46, -0.2, 5.0]]
Real result is: [ 2.5844e+02  2.2030e+01  3.5450e+01 -2.0000e-01  5.0000e+00]
Predict result is:  [[259.85499999999996, 22.018000000000004, 35.48, -0.2, 5.0]]
Real result is: [ 2.5987e+02  2.2020e+01  3.

Predict result is:  [[334.62460000000004, 21.7572, 35.12, -0.07, 5.0]]
Real result is: [ 3.3463e+02  2.1750e+01  3.5100e+01 -7.0000e-02  5.0000e+00]
Predict result is:  [[336.02920000000006, 21.7544, 35.11, -0.07, 5.0]]
Real result is: [ 3.3604e+02  2.1750e+01  3.5100e+01 -6.0000e-02  5.0000e+00]
Predict result is:  [[337.4338000000001, 21.7516, 35.12, -0.07, 5.0]]
Real result is: [ 3.3743e+02  2.1750e+01  3.5090e+01 -6.0000e-02  5.0000e+00]
Predict result is:  [[338.8384000000001, 21.7488, 35.11, -0.07, 5.0]]
Real result is: [ 3.3883e+02  2.1750e+01  3.5090e+01 -6.0000e-02  5.0000e+00]
Predict result is:  [[340.2430000000001, 21.746, 35.12, -0.07, 5.0]]
Real result is: [ 3.4024e+02  2.1740e+01  3.5090e+01 -6.0000e-02  5.0000e+00]
Predict result is:  [[336.0338, 21.7472, 35.09, -0.07, 5.0]]
Real result is: [ 3.3604e+02  2.1750e+01  3.5100e+01 -6.0000e-02  5.0000e+00]
Predict result is:  [[337.4378, 21.7444, 35.11, -0.07, 5.0]]
Real result is: [ 3.3743e+02  2.1750e+01  3.5090e+01 -6.000

In [126]:
right = [0, 0, 0, 0, 0]
error = [0, 0, 0, 0, 0]
for i in range(1000):
    # 'x', 'y', 'xVelocity', 'yVelocity', 'state'
    current_data = [train_data6[i+2]]
    history_data1 = [train_data6[i]]
    history_data2 = [train_data6[i+1]]
    for j in range(5):        
        predicted_state = model6c.predict(current_data)
        #print('predict state is:',predicted_state)
        posterior_probs = model6c.predict_proba(current_data)
        most_probable_state = np.argmax(posterior_probs)
        predicted_observation = model6c.means_[most_probable_state]
        if (predicted_state == [2]):
            predict_result = current_data[0][4] 
        else:
            predict_result = predicted_observation[4]
        if(predict_result==int(train_data6[i+j+3][4])):
            right[j] = right[j] + 1
        else:
            error[j] = error[j] + 1
        
        next_vx = current_data[0][2] + ((current_data[0][2] - history_data2[0][2]) - (history_data2[0][2] - history_data1[0][2]))
        next_vy = current_data[0][3] + ((current_data[0][3] - history_data2[0][3]) - (history_data2[0][3] - history_data1[0][3]))
        next_x = 0.04 * 0.5 * (next_vx + current_data[0][2]) + current_data[0][0]
        next_y = 0.04 * 0.5 * (next_vy + current_data[0][3]) + current_data[0][1]

        current_data = [[next_x, next_y, next_vx, next_vy, predict_result]]
        history_data1 = history_data2
        history_data2 = current_data
        print('Predict result is: ', current_data)
        print('Real result is:', train_data6[i+j+3])

        
print('Right Num:', right, 'Error Num:', error)

Predict result is:  [[244.5302, 25.66, 23.250000000000004, 0.0, 7.0]]
Real result is: [244.5   25.66  23.26  -0.     7.  ]
Predict result is:  [[245.4606, 25.66, 23.27, 0.0, 7.0]]
Real result is: [245.41  25.66  23.26  -0.     7.  ]
Predict result is:  [[246.391, 25.66, 23.250000000000004, 0.0, 7.0]]
Real result is: [ 2.4633e+02  2.5660e+01  2.3260e+01 -1.0000e-02  7.0000e+00]
Predict result is:  [[247.32139999999998, 25.66, 23.27, 0.0, 7.0]]
Real result is: [ 2.4726e+02  2.5660e+01  2.3260e+01 -1.0000e-02  7.0000e+00]
Predict result is:  [[248.25179999999997, 25.66, 23.250000000000004, 0.0, 7.0]]
Real result is: [ 2.4819e+02  2.5660e+01  2.3260e+01 -1.0000e-02  7.0000e+00]
Predict result is:  [[245.4306, 25.66, 23.27, 0.0, 7.0]]
Real result is: [245.41  25.66  23.26  -0.     7.  ]
Predict result is:  [[246.3612, 25.66, 23.26, 0.0, 7.0]]
Real result is: [ 2.4633e+02  2.5660e+01  2.3260e+01 -1.0000e-02  7.0000e+00]
Predict result is:  [[247.2918, 25.66, 23.27, 0.0, 7.0]]
Real result is:

Predict result is:  [[263.05080000000004, 25.647199999999998, 23.26, -0.04, 7.0]]
Real result is: [ 2.6305e+02  2.5650e+01  2.3260e+01 -5.0000e-02  7.0000e+00]
Predict result is:  [[263.98120000000006, 25.645799999999998, 23.26, -0.03, 7.0]]
Real result is: [ 2.6398e+02  2.5650e+01  2.3260e+01 -5.0000e-02  7.0000e+00]
Predict result is:  [[264.9116000000001, 25.644399999999997, 23.26, -0.04, 7.0]]
Real result is: [ 2.6491e+02  2.5640e+01  2.3260e+01 -6.0000e-02  7.0000e+00]
Predict result is:  [[265.8420000000001, 25.642999999999997, 23.26, -0.03, 7.0]]
Real result is: [ 2.6584e+02  2.5640e+01  2.3270e+01 -6.0000e-02  7.0000e+00]
Predict result is:  [[263.0504, 25.6484, 23.26, -0.04, 7.0]]
Real result is: [ 2.6305e+02  2.5650e+01  2.3260e+01 -5.0000e-02  7.0000e+00]
Predict result is:  [[263.98080000000004, 25.6468, 23.26, -0.04, 7.0]]
Real result is: [ 2.6398e+02  2.5650e+01  2.3260e+01 -5.0000e-02  7.0000e+00]
Predict result is:  [[264.91120000000006, 25.6452, 23.26, -0.04, 7.0]]
Rea

Predict result is:  [[338.60080000000005, 25.244200000000003, 23.34, -0.04000000000000001, 6.0]]
Real result is: [ 3.3862e+02  2.5240e+01  2.3340e+01 -4.0000e-02  6.0000e+00]
Predict result is:  [[339.53440000000006, 25.242200000000004, 23.34, -0.06, 6.0]]
Real result is: [ 3.3956e+02  2.5240e+01  2.3340e+01 -3.0000e-02  6.0000e+00]
Predict result is:  [[340.4680000000001, 25.240200000000005, 23.34, -0.04000000000000001, 6.0]]
Real result is: [ 3.4049e+02  2.5240e+01  2.3340e+01 -3.0000e-02  6.0000e+00]
Predict result is:  [[337.6736, 25.2478, 23.34, -0.06, 6.0]]
Real result is: [ 3.3768e+02  2.5240e+01  2.3340e+01 -4.0000e-02  6.0000e+00]
Predict result is:  [[338.60720000000003, 25.245600000000003, 23.34, -0.05, 6.0]]
Real result is: [ 3.3862e+02  2.5240e+01  2.3340e+01 -4.0000e-02  6.0000e+00]
Predict result is:  [[339.54080000000005, 25.243400000000005, 23.34, -0.06, 6.0]]
Real result is: [ 3.3956e+02  2.5240e+01  2.3340e+01 -3.0000e-02  6.0000e+00]
Predict result is:  [[340.474400

Predict result is:  [[410.5624000000001, 25.362000000000002, 23.39, 0.08, 7.0]]
Real result is: [4.1056e+02 2.5360e+01 2.3390e+01 8.0000e-02 7.0000e+00]
Predict result is:  [[411.4980000000001, 25.365000000000002, 23.39, 0.07, 7.0]]
Real result is: [4.115e+02 2.536e+01 2.339e+01 8.000e-02 7.000e+00]
Predict result is:  [[408.6856, 25.3532, 23.39, 0.08, 7.0]]
Real result is: [4.0869e+02 2.5350e+01 2.3390e+01 8.0000e-02 7.0000e+00]
Predict result is:  [[409.62120000000004, 25.3564, 23.39, 0.08, 7.0]]
Real result is: [4.0963e+02 2.5360e+01 2.3390e+01 8.0000e-02 7.0000e+00]
Predict result is:  [[410.55680000000007, 25.3596, 23.39, 0.08, 7.0]]
Real result is: [4.1056e+02 2.5360e+01 2.3390e+01 8.0000e-02 7.0000e+00]
Predict result is:  [[411.4924000000001, 25.3628, 23.39, 0.08, 7.0]]
Real result is: [4.115e+02 2.536e+01 2.339e+01 8.000e-02 7.000e+00]
Predict result is:  [[412.4280000000001, 25.366, 23.39, 0.08, 7.0]]
Real result is: [4.1243e+02 2.5370e+01 2.3390e+01 8.0000e-02 7.0000e+00]
Pr

Predict result is:  [[227.32880000000003, 25.6118, 23.36, -0.09, 7.0]]
Real result is: [ 2.2734e+02  2.5610e+01  2.3370e+01 -1.1000e-01  7.0000e+00]
Predict result is:  [[228.26320000000004, 25.607799999999997, 23.36, -0.11000000000000001, 7.0]]
Real result is: [ 2.2829e+02  2.5600e+01  2.3370e+01 -1.1000e-01  7.0000e+00]
Predict result is:  [[229.19760000000005, 25.603799999999996, 23.36, -0.09, 7.0]]
Real result is: [ 2.2923e+02  2.5590e+01  2.3380e+01 -1.1000e-01  7.0000e+00]
Predict result is:  [[230.13200000000006, 25.599799999999995, 23.36, -0.11000000000000001, 7.0]]
Real result is: [ 2.3017e+02  2.5590e+01  2.3380e+01 -1.1000e-01  7.0000e+00]
Predict result is:  [[227.335, 25.6062, 23.380000000000003, -0.09, 7.0]]
Real result is: [ 2.2734e+02  2.5610e+01  2.3370e+01 -1.1000e-01  7.0000e+00]
Predict result is:  [[228.2698, 25.602400000000003, 23.36, -0.1, 7.0]]
Real result is: [ 2.2829e+02  2.5600e+01  2.3370e+01 -1.1000e-01  7.0000e+00]
Predict result is:  [[229.2046, 25.598600

...

Predict result is:  [[195.70520000000002, 25.705799999999996, 36.71, 0.04000000000000001, 7.0]]
Real result is: [1.9572e+02 2.5710e+01 3.6700e+01 3.0000e-02 7.0000e+00]
Predict result is:  [[197.17360000000002, 25.707799999999995, 36.71, 0.06, 7.0]]
Real result is: [1.9719e+02 2.5710e+01 3.6700e+01 3.0000e-02 7.0000e+00]
Predict result is:  [[198.64200000000002, 25.709799999999994, 36.71, 0.04000000000000001, 7.0]]
Real result is: [1.9865e+02 2.5710e+01 3.6700e+01 2.0000e-02 7.0000e+00]
Predict result is:  [[194.2384, 25.7016, 36.71, 0.039999999999999994, 7.0]]
Real result is: [1.9425e+02 2.5710e+01 3.6710e+01 4.0000e-02 7.0000e+00]
Predict result is:  [[195.70680000000002, 25.7034, 36.71, 0.05, 7.0]]
Real result is: [1.9572e+02 2.5710e+01 3.6700e+01 3.0000e-02 7.0000e+00]
Predict result is:  [[197.17520000000002, 25.705199999999998, 36.71, 0.039999999999999994, 7.0]]
Real result is: [1.9719e+02 2.5710e+01 3.6700e+01 3.0000e-02 7.0000e+00]
Predict result is:  [[198.64360000000002, 25.7

Predict result is:  [[279.08920000000006, 25.4322, 36.49, -0.11, 7.0]]
Real result is: [ 2.791e+02  2.543e+01  3.649e+01 -9.000e-02  7.000e+00]
Predict result is:  [[280.5488000000001, 25.4282, 36.49, -0.09000000000000001, 7.0]]
Real result is: [ 2.8056e+02  2.5430e+01  3.6490e+01 -8.0000e-02  7.0000e+00]
Predict result is:  [[282.0084000000001, 25.4242, 36.49, -0.11, 7.0]]
Real result is: [ 2.8203e+02  2.5430e+01  3.6490e+01 -7.0000e-02  7.0000e+00]
Predict result is:  [[283.46800000000013, 25.420199999999998, 36.49, -0.09000000000000001, 7.0]]
Real result is: [ 2.8348e+02  2.5420e+01  3.6500e+01 -7.0000e-02  7.0000e+00]
Predict result is:  [[279.0896, 25.4264, 36.49, -0.08999999999999998, 7.0]]
Real result is: [ 2.791e+02  2.543e+01  3.649e+01 -9.000e-02  7.000e+00]
Predict result is:  [[280.54920000000004, 25.422600000000003, 36.49, -0.1, 7.0]]
Real result is: [ 2.8056e+02  2.5430e+01  3.6490e+01 -8.0000e-02  7.0000e+00]
Predict result is:  [[282.00880000000006, 25.418800000000005, 

Predict result is:  [[338.9999999999999, 25.486200000000004, 36.63, 0.09, 7.0]]
Real result is: [3.3905e+02 2.5490e+01 3.6650e+01 1.0000e-01 7.0000e+00]
Predict result is:  [[340.46499999999986, 25.490200000000005, 36.62, 0.11000000000000001, 7.0]]
Real result is: [3.4052e+02 2.5500e+01 3.6660e+01 1.0000e-01 7.0000e+00]
Predict result is:  [[336.0558, 25.4838, 36.65, 0.09, 7.0]]
Real result is: [3.3609e+02 2.5480e+01 3.6640e+01 1.0000e-01 7.0000e+00]
Predict result is:  [[337.52139999999997, 25.487599999999997, 36.63, 0.1, 7.0]]
Real result is: [3.3757e+02 2.5490e+01 3.6650e+01 1.0000e-01 7.0000e+00]
Predict result is:  [[338.98699999999997, 25.491399999999995, 36.65, 0.09, 7.0]]
Real result is: [3.3905e+02 2.5490e+01 3.6650e+01 1.0000e-01 7.0000e+00]
Predict result is:  [[340.45259999999996, 25.495199999999993, 36.63, 0.1, 7.0]]
Real result is: [3.4052e+02 2.5500e+01 3.6660e+01 1.0000e-01 7.0000e+00]
Predict result is:  [[341.91819999999996, 25.49899999999999, 36.65, 0.09, 7.0]]
Real 

Predict result is:  [[389.0508, 25.6236, 36.77, 0.09, 7.0]]
Real result is: [3.8905e+02 2.5620e+01 3.6770e+01 9.0000e-02 7.0000e+00]
Predict result is:  [[390.5216, 25.6272, 36.77, 0.09, 7.0]]
Real result is: [3.9053e+02 2.5620e+01 3.6770e+01 9.0000e-02 7.0000e+00]
Predict result is:  [[391.9924, 25.630799999999997, 36.77, 0.09, 7.0]]
Real result is: [3.920e+02 2.563e+01 3.677e+01 9.000e-02 7.000e+00]
Predict result is:  [[393.4632, 25.634399999999996, 36.77, 0.09, 7.0]]
Real result is: [3.9347e+02 2.5630e+01 3.6770e+01 9.0000e-02 7.0000e+00]
Predict result is:  [[394.93399999999997, 25.637999999999995, 36.77, 0.09, 7.0]]
Real result is: [3.9493e+02 2.5630e+01 3.6770e+01 9.0000e-02 7.0000e+00]
Predict result is:  [[390.5208, 25.6236, 36.77, 0.09, 7.0]]
Real result is: [3.9053e+02 2.5620e+01 3.6770e+01 9.0000e-02 7.0000e+00]
Predict result is:  [[391.9916, 25.6272, 36.77, 0.09, 7.0]]
Real result is: [3.920e+02 2.563e+01 3.677e+01 9.000e-02 7.000e+00]
Predict result is:  [[393.4624, 25.6

Predict result is:  [[67.95299999999999, 26.380000000000006, 33.22, -0.05, 7.0]]
Real result is: [ 6.789e+01  2.638e+01  3.318e+01 -6.000e-02  7.000e+00]
Predict result is:  [[63.9478, 26.388, 33.190000000000005, -0.05, 7.0]]
Real result is: [ 6.393e+01  2.638e+01  3.320e+01 -5.000e-02  7.000e+00]
Predict result is:  [[65.2758, 26.386000000000003, 33.21, -0.05, 7.0]]
Real result is: [ 6.525e+01  2.638e+01  3.319e+01 -5.000e-02  7.000e+00]
Predict result is:  [[66.6038, 26.384000000000004, 33.190000000000005, -0.05, 7.0]]
Real result is: [ 6.657e+01  2.638e+01  3.319e+01 -5.000e-02  7.000e+00]
Predict result is:  [[67.93180000000001, 26.382000000000005, 33.21, -0.05, 7.0]]
Real result is: [ 6.789e+01  2.638e+01  3.318e+01 -6.000e-02  7.000e+00]
Predict result is:  [[69.25980000000001, 26.380000000000006, 33.190000000000005, -0.05, 7.0]]
Real result is: [ 6.921e+01  2.638e+01  3.318e+01 -6.000e-02  7.000e+00]
Predict result is:  [[65.2582, 26.378, 33.21, -0.05, 7.0]]
Real result is: [ 6.